In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=50f6488bf4773cdf97a136334937382f0899a29213f35e6251ef869c5277abe3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit



# Khởi tạo một phiên Spark
spark = SparkSession.builder.appName("TextClassification").getOrCreate()

# Tạo tập dữ liệu huấn luyện
train_data = [
    ("Tôi bị bệnh đau đầu, chóng mặt", 1),
    ("tôi muốn mua áo mới", 0),
    ("tôi cần uống thuốc do tôi bị bệnh mãn tính", 1),
    ("tôi cần học bài nếu bởi vì John không chịu giúp đỡ tôi", 3),
    ("Tôi muốn mua thực phẩm để nấu ăn", 0),
    ("tôi cần một tài xế để đưa tôi đến bệnh viện", 3),
    ("Tôi muốn mua một chiếc điện thoại mới", 0),
    ("tôi đau bên cánh tay và cần tham khảo ý kiến bác sĩ", 1),
    ("tôi cần thuê một người để làm việc nhà", 3),
    ("Tôi muốn mua một vé máy bay", 0),
    ("tôi cần sửa chữa máy tính của tôi", 3),
    ("tôi bị sốt và ho cả ngày, cần mua thuốc", 1),
    ("tôi muốn mua một cuốn sách mới", 0),
    ("tôi cần học cách nấu món ăn mới", 3),
    ("tôi cần người để sửa máy lạnh", 3),
    ("tôi đang sốt và có triệu chứng đau họng", 1),
    ("tôi muốn mua một bộ quần áo thể thao mới", 0),
    ("tôi cần học tiếng Anh để du lịch", 3),
    ("tôi muốn mua một món quà cho ngày sinh nhật của bạn", 0),
    ("tôi đang ho và có triệu chứng viêm họng", 1),
    ("tôi cần thuê một lái xe để đưa tôi đến sân bay", 3),
    ("tôi muốn mua một cặp giày thể thao", 0),
    ("tôi đang sốt và mệt mỏi", 1),
    ("tôi cần người giúp việc trong nhà", 3),
    ("tôi muốn mua một điện thoại di động mới", 0),
    ("tôi cần học làm bánh để chuẩn bị cho bữa tiệc", 3),
    ("tôi muốn mua một chiếc đồng hồ mới", 0),
    ("tôi cần người để vận chuyển hàng hóa", 3),
    ("tôi muốn mua một cái laptop mới", 0),
    ("tôi đang bị đau bên hông và cần xem bác sĩ", 1),
    ("tôi cần thuê một chuyên gia sửa máy tính", 3),
    ("tôi muốn mua vé máy bay đi du lịch", 0),
    ("tôi đang thất nghiệp và cần tìm việc làm", 3),
    ("tôi cần mua thức ăn cho mèo cưng", 0),
    ("tôi muốn mua một cuốn sách mới", 0),
    ("tôi cần người giúp việc làm sạch nhà cửa", 3),
    ("tôi đang có triệu chứng sốt cao và ho khan", 1),
    ("tôi cần thuê một lái xe đưa tôi đến sân bay", 3),
]

# Tạo DataFrame từ tập dữ liệu huấn luyện
train_df = spark.createDataFrame(train_data, ["text", "label"])

# Tiền xử lý dữ liệu
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Tạo một pipeline
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, vectorizer, lr])

# Huấn luyện mô hình
model = pipeline.fit(train_df)

# Tập dữ liệu cần dự đoán
test_data = [
    "tôi cần mua áo và máy tính cho ngày mai",
    "tôi đang bị sốt cao và không biết khi nào thì đau đầu mới hết",
    "tôi đang khỏe hơn sau dần 3 ngày bị bệnh",
    "tôi đang thất nghiệp sau hai tháng làm việc",
]

# Tạo DataFrame từ tập dữ liệu dự đoán
test_df = spark.createDataFrame([(text, 0) for text in test_data], ["text", "label"])

# Dự đoán
predictions = model.transform(test_df)

# Hiển thị kết quả
predictions.select("text", "prediction").show()



+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|tôi cần mua áo và...|       0.0|
|tôi đang bị sốt c...|       1.0|
|tôi đang khỏe hơn...|       1.0|
|tôi đang thất ngh...|       3.0|
+--------------------+----------+

